In [15]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools, AllChem
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams
import random

# Adjust the display.max_columns option in pd to display all columns
pd.set_option('display.max_columns', None)

In [3]:
data_path = '/Users/hek/OneDrive - Eastern Connecticut State University/Research_Related/Cheminformatics Projects/Project 4. Deep learning, Bioactivity of small molecule/ChEMBL Datasets/'
decoy_path = '/Users/hek/OneDrive - Eastern Connecticut State University/Research_Related/Cheminformatics Projects/DUDE database/Decoys/'
decoys = pd.read_csv(decoy_path+"Merged DUD Decoys with canonical smiles.csv")
target_path = '/Users/hek/OneDrive - Eastern Connecticut State University/Research_Related/Cheminformatics Projects/Project 4. Deep learning, Bioactivity of small molecule/Target Information/'
targets = pd.read_csv(target_path+"AD targets_ID.csv")

### Step I. Filter dataset with the multiple criteria:
Filters:
* step 1: Human targets (Homo sapiens), single protein (target confidence score: 9) 
* step 2: Only standard potency measurements (for example: EC50, IC50, Kd, Ki) were considered 
* step 3: All compounds annotated as (‘inactive’, ‘not active’, ‘inconclusive’, ‘potential transcription error’, or ‘pan assay interference compounds (PAINS)’) were discarded.
* step 4: Only compounds with reported direct interactions (target relationship type: “B”)
* setp 5: Exact activity measurements (“=”)
* step 6: Molecular weight =< 1000 Da


* step 7: **PAINS filter**
"Pan Assay Interference Compounds": sets of substructures that appears as frequent hitters in many HTS campaign. 

PAINS show activity at numerous targets rather than one specific target. Such behavior results from unspecific binding or interaction with assay components. 

The PAINS filter is already implemented in RDKit. Such pre-defined filters can be applied via the FilterCatalog class.

**Input**: 
df == Original ChEMBL dataset

**Output**: 
filter_df

In [4]:
# For step 2. 
# Use groupby analysis to identify "Standard Type" with only numerical "pChEMBL Value"
# Return a list of selected "Standard Type"
def Select_Standard_Type(DF):
    """
    Input: Original DF
    ------------------
    Output: A list of activity 'Standard Type'
    """
    # group the dataframe by type and inspect the pchembl_value column
    Grouped = DF.groupby('Standard Type')['pChEMBL Value']

    Standard_Type = []
    
    # check if any group has only numerical values or only None values
    for group_name, group_values in Grouped:
        group_values = group_values.dropna()
        if len(group_values) == 0:
            print(f"All values in group {group_name} are None.")
        elif all(isinstance(val, float) for val in group_values):
            print(f"All values in group {group_name} are numerical.")
            Standard_Type.append(group_name)
        else:
            print(f"Group {group_name} has mixed data types.")
        
    print("Standard potency measurement types to keep are:", Standard_Type)
    
    return Standard_Type

In [5]:
def PAINS_Filter(Filtered_DF):
    # Add molecule column
    PandasTools.AddMoleculeColumnToFrame(Filtered_DF, smilesCol="Smiles")
    
    # Initialize filter
    params = FilterCatalogParams()
    params.AddCatalog(FilterCatalogParams.FilterCatalogs.PAINS)
    catalog = FilterCatalog(params)
    
    # Iterate over the dataframe and apply PAINS filter
    matches = []
    clean = []
    for index, row in Filtered_DF.iterrows():
        mol = row.ROMol
        entry = catalog.GetFirstMatch(mol) #Get the first matching PAINS
    
        if entry is not None:
            # Store PAINS information
            matches.append(
                {"chembl_id": index,
                "rdkit_molecule": mol,
                "pains": entry.GetDescription().capitalize(),
                }
            )
        else:
            # Collect indices of molecules without PAINS
            clean.append(index)

    matches = pd.DataFrame(matches)
    
    # Keep molecules without PAINS
    Filtered_DF = Filtered_DF.loc[clean]

    # CHECK_OUTPUT
    print(f"Number of compounds with PAINS: {len(matches)}")
    print(f"Number of compounds without PAINS: {len(clean)}")
    
    Filtered_DF = Filtered_DF.drop("ROMol", axis=1)
    return Filtered_DF

In [6]:
# Step 1 - 7
# This function read in the original dataset, then return the filtered dataset with unique compounds. 
# Additionally, also clean out the smiles of all unique compounds. 
def Get_Filtered_Df(DF):
    # Print out the first 20 rows of the original dataset
    print("*"*20,"Original bioactivity dataset", "*"*20)
    print(DF.loc[:,['Molecular Weight','Standard Type','pChEMBL Value','Comment','Assay Type']].head(20))
    print("Number of bioactivity data points:", DF.shape[0])
    print("Number of unique compounds:",DF['Molecule ChEMBL ID'].nunique())
    targets.loc[target_idx,"N_Uniques"] = DF['Molecule ChEMBL ID'].nunique()
    print("\n")
    
    # For step 6
    # Convert 'Molecular Weight' to float datatype, since we need to filter all compounds that are too large
    DF['Molecular Weight']=pd.to_numeric(DF['Molecular Weight'], errors='coerce')
    DF['Molecular Weight'].replace('None', np.nan, inplace=True)
    DF['Molecular Weight'] = DF['Molecular Weight'].astype(float)  
    
    # For step 3
    # Convert any comment strings contain one or more digits and nothing else to NaN, then only keep NaN comments
    DF['Comment'] = DF['Comment'].replace(to_replace=r'^\d+$', value=np.nan, regex=True)
    DF['Comment'] = DF['Comment'].replace(to_replace=['Active','active'], value=np.nan, regex=True)

    # For step 2
    # Identify the 'Standard Type' that has only numerical 'pChEMBL Value' and keep only those
    print("**Calling function Select_Standard_Type**")
    Standard_Type = Select_Standard_Type(DF)
    
    # Apply the data filters 
    Filtered_DF = DF[DF['Comment'].isna()] # Step 3: Drop any non NaN comments 
    Filtered_DF = Filtered_DF[Filtered_DF['Assay Type']=='B'] # Step 4: Keep only 'B' Assay Type
    Filtered_DF = Filtered_DF[Filtered_DF['Standard Relation']=="'='"] # Step 5 
    Filtered_DF = Filtered_DF.dropna(subset=['pChEMBL Value']) # Step 7: Drop any NaN pChEMBL Values
    Filtered_DF = Filtered_DF[Filtered_DF['Standard Type'].isin(Standard_Type)] # Step 2
    
    Filtered_DF = Filtered_DF[Filtered_DF['Molecular Weight']<=1000] # Step 6
    
    # Print out the first 20 rows of the filtered dataset with above dataframe sub settings 
    print("\n")
    print("*"*20,"Filtered bioactivity dataset", "*"*20)
    print(Filtered_DF.loc[:,['Molecular Weight','Standard Type','pChEMBL Value','Comment','Assay Type']].head(20))
    print("Number of bioactivity data points:",Filtered_DF.shape[0])
    print("Number of unique compounds:",Filtered_DF['Molecule ChEMBL ID'].nunique())
   
    # Drop any row without 'Smiles'
    print("\n")
    print("*"*20,"Process and clean SMILES", "*"*20)
    Filtered_DF = Filtered_DF.dropna(subset=['Smiles'])
    
    # For molecule that has "." in the "Smiles" string, split the "Smiles" and keep the compound part
    Filtered_DF['Smiles']= Filtered_DF['Smiles'].apply(lambda x: max(x.split("."),key=len))
    print("There are", sum(Filtered_DF['Smiles'].str.find('.') != -1), "records with '.' in their canonical smiles string")
    print(f"Number of bioactivity data points: {Filtered_DF.shape[0]}")
    print(f"Number of unique compounds: {Filtered_DF['Molecule ChEMBL ID'].nunique()}")
    
    # For step 7 PAINS filter 
    print("\n")
    print("**Calling function PAINS_Filter**")
    Filtered_DF = PAINS_Filter(Filtered_DF)
    print(f"Number of bioactivity data points: {Filtered_DF.shape[0]}")
    print(f"Number of unique compounds: {Filtered_DF['Molecule ChEMBL ID'].nunique()}")
    
    return Filtered_DF

### Step II. Create final dataset using two methods

#### Method 1. Directly annotate according to pChEMBL Value
* Actives - pChEMBL Value >= 6 (equivalent to Standard Value <=1000 nM)
   -- targets.loc[target_idx,"N_Actives"]
* Inactives - pChEMBL Value <= 5 (equivalent to Standard Value >= 10000 nM)
    -- targets.loc[target_idx,"N_Inactives"]
* Compounds with pChEMBL Value > 5 and < 6 are discarded

saved to "target_idx_annotated.csv"

#### Method 2. Obtain 4 decoys per each active from DUD-E
* Inactives:
* Step 1 Random sample N_Actives*5 decoys from DUD-E
* Step 2 Calculate Active against all decoys Tanimoto coefficient;
* Step 3 Discard any decoys where Tani >=0.9
* Step 4 Repeat for all Actives
* Step 5 Keep N_Actives*4 decoys

saved to "target_idx_decoys.csv"

In [9]:
# For STEP II. 
# Some compounds have multiple bioactivity data reported. 
# The mean and std (standard deviation) of pChEMBL Values are calculated for each unique compound, any compound with disagreeing pChEMBL Values (large std) are discarded
# 'mean' is merged to the dataset
def Mean_Std_pChEMBL_Value(Filtered_DF):
    print("Number of unique compounds: ",Filtered_DF['Molecule ChEMBL ID'].nunique())
    
    # Group by "Molecule ChEMBL ID" and calculate average and standard deviation of "pChEMBL Value"
    Grouped_Filtered_DF = Filtered_DF.groupby('Molecule ChEMBL ID')['pChEMBL Value'].agg(['mean', 'std'])
    
    # The number of unique compound with std of pChEMBL Value greater than 2.0
    n_drop = Grouped_Filtered_DF[Grouped_Filtered_DF['std']>2.0].shape[0]
    print("There are",n_drop, "compounds with std pChEMBL Value > 2.0")
    
    # Get the "Molecule ChEMBL ID" with a std pChEMBL Value <= 2.0
    Remaining_IDs = Grouped_Filtered_DF[Grouped_Filtered_DF['std'].isna() | (Grouped_Filtered_DF['std']<=2.0)].index.tolist()
    print("Number of unique compounds to keep: ",len(Remaining_IDs))
    
    # Merged Filtered_DF and Grouped_Filtered_DF by "Molecule ChEMBL ID" and add mean pChEMBL 
    Merged_DF = pd.merge(Filtered_DF, Grouped_Filtered_DF[['mean']], left_on='Molecule ChEMBL ID', right_index=True, how='left')
    # Keep only "Molecule ChEMBL ID" with std pChEMBL Value <= 2.0 according to Remaining_IDs
    Merged_DF_subset = Merged_DF.set_index('Molecule ChEMBL ID').loc[Remaining_IDs]
    # Drop duplicates based on the index ('Molecule ChEMBL ID')
    Merged_DF_unique = Merged_DF_subset[~Merged_DF_subset.index.duplicated(keep='first')]

    return Merged_DF_unique

In [145]:
def Get_Decoys(Active_Smiles, Decoy_Smiles):
    print(f"Number actives {len(Active_Smiles)}")
    # Convert active smiles to RDKit 'Mol' objects
    Active_mols = [Chem.MolFromSmiles(Smiles) for Smiles in Active_Smiles]
    Active_fps = [AllChem.GetMorganFingerprint(mol,2) for mol in Active_mols]
    
    # Random sample 5*num_actives of decoys
    Sample_Decoys = random.sample(Decoy_Smiles.tolist(), len(Active_Smiles)*5)
    Decoy_mols = [Chem.MolFromSmiles(Smiles) for Smiles in Sample_Decoys]
    Decoy_fps = [AllChem.GetMorganFingerprint(mol,2) for mol in Decoy_mols]

    sm_list = []
    for Decoy_fp in Decoy_fps:
        sm_list.append(DataStructs.BulkTanimotoSimilarity(Decoy_fp, Active_fps))
    SM_DF = pd.DataFrame(sm_list)
    SM_DF["Smiles"] = Sample_Decoys
    SM_DF = SM_DF.set_index('Smiles')
    
    SM_DF = SM_DF[SM_DF.max(axis=1) <0.9]
    Decoy_list = random.sample(SM_DF.index.to_list(), len(Active_Smiles)*4)
    return Decoy_list

In [161]:
def Get_Morgan_fp(Curated_DF):
    PandasTools.AddMoleculeColumnToFrame(Curated_DF, smilesCol="Smiles")
    Curated_DF["morgan"] = Curated_DF.ROMol.apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=1024))
    Curated_DF["morganfp"] = Curated_DF["morgan"].apply(lambda x: x.ToBitString())
    Curated_DF = Curated_DF.drop(['ROMol','morgan'],axis=1)
    return Curated_DF

In [184]:
def Create_Final_Dataset(Filtered_DF,thres1=6.0,thres2=5.5):
    """
    Paremeters:
    Filtered_DF, 
    thres1= actives pChEMBL Value threshold, 
    thres2 = inactive pChEMBL Valuethreshold
    """
    # Clean up multiple pChEMBL Values
    print("**Calling function Mean_Std_pChEMBL_Value**")
    Filtered_DF_mean_pChembl_Value = Mean_Std_pChEMBL_Value(Filtered_DF)
    print("*"*20,"Filtered DF", "*"*20)
    print(Filtered_DF_mean_pChembl_Value.loc[:,['Molecular Weight','Standard Type','pChEMBL Value','mean']].head(5))
    
    print("\n")
    print("*"*20,"Creating Annotated dataset using Method 1", "*"*20)
    # Actives are compounds with mean pChEMBL Value >=6.0, Annotated as "1"
    Active_subset = Filtered_DF_mean_pChembl_Value[Filtered_DF_mean_pChembl_Value['mean']>=thres1].copy()
    Active_subset['label']=1
    targets.loc[target_idx,"N_Actives"] = Active_subset.shape[0]
    # Inactives are compounds with mean pChEMBL Value <=5.5
    Inactive_subset = Filtered_DF_mean_pChembl_Value[Filtered_DF_mean_pChembl_Value['mean']<=thres2].copy()
    Inactive_subset['label']=0
    targets.loc[target_idx,"N_Inactives"] = Inactive_subset.shape[0]
    print(f"Active/Inactive Ratio is {Active_subset.shape[0]/Inactive_subset.shape[0]}")
    Annotated_DF = pd.concat([Active_subset,Inactive_subset]).loc[:,["Smiles","label"]]
    print(f"Size of Method I dataset {Annotated_DF.shape[0]}")
    
    print("\n")
    print("*"*20,"Creating Decoyed dataset using Method 2", "*"*20)
    print("**Calling function Get_Decoys")
    Decoy_Smiles = Get_Decoys(Active_subset["Smiles"], decoys['Canonical smiles'])
    targets.loc[target_idx,"Decoys"] = Active_subset.shape[0]*4
    Decoy_subset = pd.DataFrame(Decoy_Smiles, columns=['Smiles'])
    print(f"Number of sampled decoy {len(Decoy_Smiles)}")
    Decoy_subset['label']=0
    Decoy_DF = pd.concat([Active_subset,Decoy_subset]).loc[:,["Smiles","label"]]
    Decoy_DF.index.name = 'Molecule ChEMBL ID'
    # Calculate morgan (ECFP4) fingerprint
    Annotated_DF_final = Get_Morgan_fp(Annotated_DF)
    Decoy_DF_final = Get_Morgan_fp(Decoy_DF)
    return Annotated_DF_final,Decoy_DF_final

In [ ]:
for target_idx in range(targets.shape[0]):
    target_id= targets.loc[target_idx,"Uniprot_ID"]+"_"+targets.loc[target_idx,"ChEMBL_ID"]
    print(f"Reading original ChEMBL dataset {target_id}")
    df = pd.read_csv(data_path+target_id+".csv",  delimiter=';', skiprows=0, low_memory=False)
    filtered_df = Get_Filtered_Df(df)
    filtered_df.to_csv(data_path+target_id+"_filtered.csv", index=False)
    
    annotated_df, decoy_df = Create_Final_Dataset(filtered_df)
    annotated_df.to_csv(data_path+target_id+"_annotated.csv")
    decoy_df.to_csv(data_path+target_id+"_decoy.csv")
    
    targets.to_csv(target_path+"AD targets_ID.csv")

Reading original ChEMBL dataset P50406_CHEMBL3371
******************** Original bioactivity dataset ********************
    Molecular Weight Standard Type  pChEMBL Value          Comment Assay Type
0             388.56          IC50            NaN       Not Active          F
1             408.52            Ki           8.30              NaN          B
2             386.90            Ki           8.00              NaN          B
3             364.42            Ki           6.88              NaN          B
4             176.22            Ki           7.19              NaN          B
5             382.27            Ki           9.46           309762          B
6             361.85            Ki           7.72           309798          B
7             420.33            Ki           8.42              NaN          B
8             335.45            Ki           7.89              NaN          B
9             418.52            Ki           8.02              NaN          B
10            347.40 

Number of compounds with PAINS: 533
Number of compounds without PAINS: 6163
Number of bioactivity data points: 6163
Number of unique compounds: 4580
**Calling function Mean_Std_pChEMBL_Value**
Number of unique compounds:  4580
There are 22 compounds with std pChEMBL Value > 2.0
Number of unique compounds to keep:  4558
******************** Filtered DF ********************
                    Molecular Weight Standard Type  pChEMBL Value  mean
Molecule ChEMBL ID                                                     
CHEMBL100334                  371.51          IC50           6.92  6.92
CHEMBL100366                  351.41          IC50           6.11  6.11
CHEMBL100510                  421.57          IC50           7.00  7.00
CHEMBL101028                  427.49          IC50           6.00  6.00
CHEMBL101155                  385.53          IC50           5.96  5.96


******************** Creating Annotated dataset using Method 1 ********************
Active/Inactive Ratio is 1.47080745

**Calling function Select_Standard_Type**
All values in group % Ctrl are None.
All values in group % Inhibition of Control Values are None.
All values in group Activity are None.
All values in group EC50 are numerical.
All values in group ED50 are None.
All values in group Emax are None.
All values in group FC are None.
All values in group IC50 are numerical.
All values in group INH are None.
All values in group IP are None.
All values in group Inhibition are None.
All values in group K are None.
All values in group Ka are None.
All values in group Kcat are None.
All values in group Kd are numerical.
All values in group Ki are numerical.
All values in group Kic are None.
All values in group Km are None.
All values in group RFU are None.
All values in group Ratio are None.
All values in group Ratio IC50 are None.
All values in group Ratio Ki are None.
All values in group TIME are None.
All values in group k obs / 1 are None.
All values in group pIC50 are None.
Standard potency measureme

In [ ]:
targets.head(5)